## View data with ParaVIEW

ParaFoam can read netCDF files. Let's take a PIV Case and generate `N` netCDF files containing all variables for each `N` time steps. Each file only contains (`x`, `y`, `z`) coordinates as ParaFOAM cannot handle more than three dimensions.

In [ ]:
import xarray as xr
from h5rdmtoolbox import x2hdf
from h5rdmtoolbox import tutorial
from h5rdmtoolbox.x2hdf.piv2hdf.pivview import PIVMultiPlane
from h5rdmtoolbox.h5wrapper import H5PIV, H5File

In [ ]:
plane0_dir, plane1_dir, _ = tutorial.PIVview.get_multiplane_directories()
piv_case = PIVMultiPlane([plane0_dir, plane1_dir], time_information=[5, 5])
piv_case.convert()

We collect all datasets in a list:

In [ ]:
with H5File(piv_case.hdf_filename, 'r') as h5piv:
    # h5piv.dump()
    datasets = []
    nt = h5piv['time'].size
    for ds in h5piv.datasets:
        datasets.append(ds[:])

We iterate over all timesteps, create a `xr.Dataset` for it and save each time-dataset as a netCDF4 file. Loading those files into ParaFOAM let's you explore the dataset as expected.

In [ ]:
for it in range(nt):
    ds = xr.merge([ds[:,it,...].drop('time') for ds in datasets if ds.name not in ('x', 'y', 'z', 'time', 'ix', 'iy')])
    ds['piv_flags'].attrs.pop('flag_translation')
    ds.to_netcdf(f'timestep{it:02d}.nc')

Delete the created files:

In [ ]:
import pathlib
for f in pathlib.Path.cwd().glob('timestep*[0-9].nc'):
    f.unlink()